In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [3]:
os.chdir("//home//evcho//tusofia//programming_IOT//data")
day = pd.read_excel("day.xls", sheet_name="hipodruma", header=1, usecols="A:C", dtype={"O3": float, "PM10": float})
day.head()


,Date,O3,RM10
0,01.01.2012,46.447132,44.848259
1,02.01.2012,24.386621,164.782471
2,03.01.2012,23.249441,337.119385
3,04.01.2012,23.534016,465.039429
4,05.01.2012,14.516003,308.981110


In [4]:
hour = pd.read_excel("hour.xls", sheet_name="hipodruma", header=1, usecols="A,C:E,G:H", dtype={"NO": float, "NO2": float, "GSR": float, "Press": float, "UMR": float})
hour.head()

,Date,NO,NO2,AirTemp,Press,UMR
0,01.01.2012,3.48,21.09,-1.0,954.2,79.7
1,01.01.2012,1.18,16.05,-1.0,954.6,78.4
2,01.01.2012,NaN,NaN,-1.1,955.4,78.7
3,01.01.2012,0.45,13.89,-1.2,955.7,79.3
4,01.01.2012,0.37,15.44,-1.3,956.0,77.9


In [7]:
merged_df = pd.DataFrame()
for i in range(day["Date"].size):
    day_row = day.iloc[i]
    hour_row = hour.iloc[i]
    column_date = str(day.iloc[i]["Date"])
    o3_value = day.iloc[i]["O3"]
    pm10_value = day.iloc[i]["RM10"]

    hour_rows = hour[hour["Date"].astype(str).str.contains(column_date)]
    hour_rows.insert(len(hour_rows.columns), "O3", o3_value)
    hour_rows.insert(len(hour_rows.columns), "PM10", pm10_value)

    merged_df = pd.concat([merged_df, hour_rows], ignore_index=True)
clean_df = merged_df.dropna()
    
merged_df.head()

rounded_df = clean_df.round({
    "NO": 2,
    "NO2": 2,
    "O3": 2,
    "PM10": 2,
    "AirTemp": 1,
    "UMR": 1,
    "Press": 0
    
})

rounded_df.head()


,Date,NO,NO2,AirTemp,Press,UMR,O3,PM10
0,01.01.2012,3.48,21.09,-1.0,954.0,79.7,46.45,44.85
1,01.01.2012,1.18,16.05,-1.0,955.0,78.4,46.45,44.85
3,01.01.2012,0.45,13.89,-1.2,956.0,79.3,46.45,44.85
4,01.01.2012,0.37,15.44,-1.3,956.0,77.9,46.45,44.85
5,01.01.2012,0.29,13.86,-1.3,957.0,76.7,46.45,44.85


In [8]:
rounded_df.to_csv(
    "merged_data.csv",
    columns=["NO", "NO2", "AirTemp", "Press", "UMR", "O3", "PM10"],
    index=False
)

In [9]:
x = rounded_df[["AirTemp", "Press", "UMR"]]
y = rounded_df[["NO", "NO2", "O3", "PM10"]]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.3,
    random_state=42
)